In [1]:
!pip install scikit-learn pandas joblib

In [10]:
import pandas as pd
import numpy as np
from google.colab import files
import io

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
df = pd.read_csv('/content/drive/MyDrive/Sharknado/dataset_enriquecido.csv',
                    sep=',', encoding='utf-8')

In [16]:
df.head()

,timestamp,latitude,longitude,target,tipo_amostra,sst_c,proximidade_colonia_km,profundidade_m,declive_graus,clorofila_mg_m3,energia_dinamica_score
0,2023-10-06 07:00:00,37.9871,-123.1199,1,treino_positivo,16.1009,16.3084,82.7234,23.5491,4.4933,0.8397
1,2023-10-10 06:00:00,37.2410,-122.3555,1,treino_positivo,16.4733,15.1607,194.4256,32.5370,0.9684,0.6939
2,2023-08-27 09:00:00,38.0253,-123.0596,1,treino_positivo,17.3811,6.1323,168.5702,37.6719,1.9148,0.9025
3,2023-08-13 21:00:00,37.6732,-123.0139,1,treino_positivo,16.4351,15.8193,88.4324,15.1533,3.7887,0.6871
4,2023-09-24 11:00:00,37.0317,-122.2679,1,treino_positivo,15.7180,5.1750,147.5365,40.4309,2.0562,0.9194


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

In [18]:
def treinar_modelo_tubaroes(df):
    """
    Função principal para treinar o modelo.
    Recebe o DataFrame carregado como argumento.
    """
    # --- 1. Preparar os Dados para o Treino ---
    print("2. A preparar os dados para o treino do modelo...")

    features = [
        'sst_c',
        'proximidade_colonia_km',
        'profundidade_m',
        'declive_graus',
        'clorofila_mg_m3',
        'energia_dinamica_score'
    ]

    df_treino = df[df['tipo_amostra'].str.contains('treino')]
    df_teste = df[df['tipo_amostra'].str.contains('teste')]

    X_treino = df_treino[features]
    y_treino = df_treino['target']

    X_teste = df_teste[features]
    y_teste = df_teste['target']

    print(f"   - {len(df_treino)} amostras para treino.")
    print(f"   - {len(df_teste)} amostras para teste.")

    # --- 2. Criar e Treinar o Pipeline de IA ---
    print("\n3. A construir e a treinar o pipeline de Machine Learning...")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
    ])
    pipeline.fit(X_treino, y_treino)
    print("   - Treino concluído!")

    # --- 3. Avaliar a Performance do Modelo ---
    print("\n4. A avaliar a performance do modelo no conjunto de teste...")
    predicoes_classe = pipeline.predict(X_teste)
    probabilidades = pipeline.predict_proba(X_teste)[:, 1]

    precisao = accuracy_score(y_teste, predicoes_classe)
    print(f"   - Precisão (Accuracy): {precisao * 100:.2f}%")
    print("\n   - Relatório de Classificação:")
    print(classification_report(y_teste, predicoes_classe))

    # --- 4. Analisar a Importância dos Parâmetros ---
    print("\n5. Importância de cada parâmetro para o modelo:")
    importancia_features = pd.Series(pipeline.named_steps['classifier'].feature_importances_, index=features).sort_values(ascending=False)
    print(importancia_features)

    # --- 5. Salvar o Modelo no Ambiente do Colab ---
    print("\n6. A salvar o pipeline de IA treinado...")
    nome_ficheiro_pipeline = "modelo_tubarao_v1.pkl"
    joblib.dump(pipeline, nome_ficheiro_pipeline)
    print(f"   - Pipeline de IA salvo em: '{nome_ficheiro_pipeline}'")

    # --- 6. Exemplo Prático ---
    print("\n--- EXEMPLO DE USO DO MODELO ---")
    novo_local = pd.DataFrame([{'sst_c': 15.5, 'proximidade_colonia_km': 10.2, 'profundidade_m': 180.0, 'declive_graus': 30.5, 'clorofila_mg_m3': 2.1, 'energia_dinamica_score': 0.9}])
    probabilidade_avistamento = pipeline.predict_proba(novo_local)[:, 1][0]
    print("Para um novo local com as condições de exemplo:")
    print(f"\nA probabilidade de avistamento de um tubarão-branco é de: {probabilidade_avistamento * 100:.2f}%")

    # Retorna o nome do ficheiro do modelo para a próxima célula
    return nome_ficheiro_pipeline

# --- Ponto de Entrada ---
# Executa a função de treino se o DataFrame (df) foi carregado com sucesso
if df is not None:
    nome_ficheiro_modelo = treinar_modelo_tubaroes(df)
else:
    print("\nA execução foi interrompida porque o DataFrame de dados não foi carregado.")
    nome_ficheiro_modelo = None

2. A preparar os dados para o treino do modelo...
   - 100 amostras para treino.
   - 50 amostras para teste.

3. A construir e a treinar o pipeline de Machine Learning...
   - Treino concluído!

4. A avaliar a performance do modelo no conjunto de teste...
   - Precisão (Accuracy): 100.00%

   - Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        15

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50


5. Importância de cada parâmetro para o modelo:
proximidade_colonia_km    0.254464
clorofila_mg_m3           0.244446
energia_dinamica_score    0.240768
declive_graus             0.233680
profundidade_m            0.025596
sst_c                     0.001046
dtype: float64

6. A salvar o pipeline de IA treinado...
   - Pipeline de IA salvo em: 'm